# DIII Linear Weights

In [2]:
import pandas as pd
import altair as alt
import numpy as np

In [3]:
pp_data = pd.read_csv("/Users/zoeskigen/Desktop/Linear Weights/pp-23.csv")
pp_data.head()

/var/folders/mj/b8_lt8gn6cvcflsf00z0pn6r0000gn/T/ipykernel_8492/3867123029.py:1: DtypeWarning: Columns (93) have mixed types. Specify dtype option on import or set low_memory=False.
  pp_data = pd.read_csv("/Users/zoeskigen/Desktop/Linear Weights/pp-23.csv")


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2022-02-04,14:03:11.73,1,1,"Deliyannis, Harry",1000115586,Right,POM_COL,"Savant, Jack",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2022-02-04,14:03:31.35,1,2,"Deliyannis, Harry",1000115586,Right,POM_COL,"Savant, Jack",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2022-02-04,14:04:02.74,1,3,"Deliyannis, Harry",1000115586,Right,POM_COL,"Savant, Jack",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
3,4,2022-02-04,14:04:19.48,1,4,"Deliyannis, Harry",1000115586,Right,POM_COL,"Savant, Jack",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
4,5,2022-02-04,14:04:35.49,1,5,"Deliyannis, Harry",1000115586,Right,POM_COL,"Savant, Jack",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


# Step 1: Adjust the Simulation Function
Modify the simulation function to format the base states as specified and include the number of outs directly in the 'BaseOutState' string.

In [7]:
def update_bases_and_outs(bases, result, outs_on_play, current_outs):
    # Handle base movements
    if result == 'Single':
        new_bases = [True] + bases[:2]
    elif result == 'Double':
        new_bases = [False, True] + bases[:1]
    elif result == 'Triple':
        new_bases = [False, False, True]
    elif result == 'HomeRun':
        new_bases = [False, False, False]
    else:
        new_bases = bases  # Default to no base change for other results

    # Update outs
    current_outs += outs_on_play

    # Reset outs and bases if inning ends
    if current_outs >= 3:
        current_outs = 0
        new_bases = [False, False, False]

    return new_bases, current_outs

def simulate_game(pp_data):
    current_bases = [False, False, False]
    current_outs = 0
    base_out_states = []

    for index, row in pp_data.iterrows():
        current_bases, current_outs = update_bases_and_outs(current_bases, row['PlayResult'], row['OutsOnPlay'], current_outs)
        base_state = ' '.join(['1' if base else '_' for base in current_bases])
        base_out_state = f"{base_state}, {current_outs} Outs"
        base_out_states.append(base_out_state)

    return base_out_states

# Apply the function to simulate the game
pp_data['BaseOutState'] = simulate_game(pp_data)

# Step 2: Calculate Run Expectancy for Each State

In [8]:
def calculate_run_expectancy(group):
    total_runs = group['RunsScored'].sum()  # Ensure you have a way to track runs scored
    total_instances = group.shape[0]
    return total_runs / total_instances

grouped_data = pp_data.groupby('BaseOutState').apply(calculate_run_expectancy).reset_index(name='RunExpectancy')

/var/folders/mj/b8_lt8gn6cvcflsf00z0pn6r0000gn/T/ipykernel_8492/3281834268.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = pp_data.groupby('BaseOutState').apply(calculate_run_expectancy).reset_index(name='RunExpectancy')


# Step 3: Pivot Data to Create Matrix

In [9]:
matrix = grouped_data.pivot_table(index='BaseOutState', values='RunExpectancy', aggfunc='first')
matrix

,RunExpectancy
BaseOutState,
"1 1 1, 0 Outs",0.386364
"1 1 1, 1 Outs",0.111801
"1 1 1, 2 Outs",0.089674
"1 1 _, 0 Outs",0.062762
"1 1 _, 1 Outs",0.065823
"1 1 _, 2 Outs",0.057018
"1 _ 1, 0 Outs",0.225806
"1 _ 1, 1 Outs",0.138614
"1 _ 1, 2 Outs",0.050898


### Run Values

In [10]:
pp_data = pp_data[pp_data['PlayResult'] != 'undefined']

# Assuming 'state' is the column in 'matrix' that corresponds to base-out states
pp_data = pp_data.merge(matrix, left_on='start_base_out_state', right_on='state', how='left')
pp_data.rename(columns={'RE': 'RE_start'}, inplace=True)
pp_data = pp_data.merge(matrix, left_on='end_base_out_state', right_on='state', how='left')
pp_data.rename(columns={'RE': 'RE_end'}, inplace=True)

KeyError: 'state'

In [11]:
print(matrix.columns)

Index(['RunExpectancy'], dtype='object')
